In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/london-bike-sharing-dataset/london_merged.csv


In [2]:
import pandas as pd

In [3]:
# read in csv file as pandas dataframe
bikes = pd.read_csv('/kaggle/input/london-bike-sharing-dataset/london_merged.csv')

In [4]:
# explore data
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [5]:
# display number of columns, rows
bikes.shape

(17414, 10)

In [6]:
# visualize data
bikes

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...
17409,2017-01-03 19:00:00,1042,5.0,1.0,81.0,19.0,3.0,0.0,0.0,3.0
17410,2017-01-03 20:00:00,541,5.0,1.0,81.0,21.0,4.0,0.0,0.0,3.0
17411,2017-01-03 21:00:00,337,5.5,1.5,78.5,24.0,4.0,0.0,0.0,3.0
17412,2017-01-03 22:00:00,224,5.5,1.5,76.0,23.0,4.0,0.0,0.0,3.0


In [7]:
# count unique weather_code values
bikes.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [8]:
# count unique season values
bikes.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [9]:
# specify new column names
new_columns_dictionary = {
    'timestamp':'Time',
    'cnt':'Count',
    't1':'Temperature_Real_C',
    't2':'Temperature_FeelsLike_C',
    'hum':'Humidity_Percent',
    'wind_speed':'Wind_Speed_Kph',
    'weather_code':'Weather',
    'is_holiday':'Is_Holiday',
    'is_weekend':'Is_Weekend',
    'season':'Season',
}

# apply new column names
bikes.rename(new_columns_dictionary, axis=1, inplace=True)

In [10]:
# set humidity values to a percentage (b/w 0-1)
bikes.Humidity_Percent = bikes.Humidity_Percent / 1000

In [11]:
# create season dictionary
season_dictionary = {
    '0.0':'Spring',
    '1.0':'Summer',
    '2.0':'Autmn',
    '3.0':'Winter',
}
# create weather dictionary
weather_dictionary = {
    '1.0':'Clear',
    '2.0':'Scattered clouds',
    '3.0':'Broken clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorm',
    '26.0':'Snow',
}

# change season column to string data type
bikes.Season = bikes.Season.astype('str')
# map season values to string data type
bikes.Season = bikes.Season.map(season_dictionary)

# change weather column to string data type
bikes.Weather = bikes.Weather.astype('str')
# map weather values to string data type
bikes.Weather = bikes.Weather.map(weather_dictionary)

In [12]:
# check dataframe to verify mappings updated
bikes.head()

,Time,Count,Temperature_Real_C,Temperature_FeelsLike_C,Humidity_Percent,Wind_Speed_Kph,Weather,Is_Holiday,Is_Weekend,Season
0,2015-01-04 00:00:00,182,3.0,2.0,0.0930,6.0,Broken clouds,0.0,1.0,Winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.0930,5.0,Clear,0.0,1.0,Winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.0965,0.0,Clear,0.0,1.0,Winter
3,2015-01-04 03:00:00,72,2.0,2.0,0.1000,0.0,Clear,0.0,1.0,Winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.0930,6.5,Clear,0.0,1.0,Winter


In [13]:
# export final dataframe to excel file for Tableau Visualization
bikes.to_excel('london_bikes_final.xlsx', sheet_name = 'BikeData')